In [1]:
using Korg
using Trapz

In [4]:
"""
    linelist_neighbourhood_indices(linelist, ew_window_size)

Group lines together such that no two lines are closer than twice the value of `line_buffer`.

# Arguments:
- `linelist`: A vector of [`Line`](@ref)s (see [`read_linelist`](@ref), 
   [`get_APOGEE_DR17_linelist`](@ref), and [`get_VALD_solar_linelist`](@ref)).
- `ew_window_size`: the minimum separation (in Å) either side of lines in a group

# Returns
A vector of vectors, where each inner vector contains the indices of lines in a group.
"""
function linelist_neighbourhood_indices(linelist, ew_window_size)
    linelist_neighbourhood_indices = []        
    current_group = [1]    
    ew_window_size_overlap_cm = 2 * 1e-8 * ew_window_size
    for i in 2:length(linelist)
        if (linelist[i].wl - linelist[current_group[end]].wl) > ew_window_size_overlap_cm
            push!(current_group, i)
        else
            push!(linelist_neighbourhood_indices, current_group)
            current_group = [i]  
        end
    end
    push!(linelist_neighbourhood_indices, current_group)
    linelist_neighbourhood_indices
end

linelist_neighbourhood_indices

In [58]:

"""
    ews_to_abundances(atm, linelist, A_X, ews; kwargs... )

Compute per-line abundances given a model atmosphere and a list of lines with equivalent widths.

# Arguments:
- `atm`: the model atmosphere (see [`read_model_atmosphere`](@ref))
- `linelist`: A vector of [`Line`](@ref)s (see [`read_linelist`](@ref), 
   [`get_APOGEE_DR17_linelist`](@ref), and [`get_VALD_solar_linelist`](@ref)).
- `A_X`: a vector containing the A(X) abundances (log(X/H) + 12) for elements from hydrogen to 
  uranium.  (see [`format_A_X`](@ref))
- `ews`: a vector of equivalent widths (in mÅ)

# Returns
A vector of abundances (log10(n_X/n_H) + 12 format) for each line in `linelist`.

# Optional arguments:
- `vmic` (default: 1.0) is the microturbulent velocity, ``\\xi``, in km/s.
- `ew_window_size` (default: 2): the farthest (in Å) to consider equivalent width contributions for any line.
- `air_wavelengths` (default: `false`): Whether or not the input wavelengths are air wavelenths to 
   be converted to vacuum wavelengths by Korg.  The conversion will not be exact, so that the 
   wavelenth range can internally be represented by an evenly-spaced range.  If the approximation 
   error is greater than `wavelength_conversion_warn_threshold`, an error will be thrown. (To do 
   wavelength conversions yourself, see [`air_to_vacuum`](@ref) and [`vacuum_to_air`](@ref).)
- `wavelength_conversion_warn_threshold` (default: 1e-4): see `air_wavelengths`. (In Å.)
"""
function ews_to_abundances(atm, linelist, A_X, ews, ew_window_size::Real=5.0, λ_step=0.01; synthesize_kwargs...)

    synthesize_kwargs = Dict(synthesize_kwargs)
    if get(synthesize_kwargs, :hydrogen_lines, false)
        throw(ArgumentError("hydrogen_lines must be disabled"))
    end
    print(synthesize_kwargs)

    if !issorted(linelist; by=l->l.wl) 
        throw(ArgumentError("linelist must be sorted"))
    end

    if any(l -> Korg.ismolecule(l.species), linelist)
        throw(ArgumentError("linelist contains molecular species"))
    end

    # Check that the user is supplying EWs in mA
    if 1 > maximum(ews)
        @warn "Maximum EW given is less than 1 mA. Check that you're giving EWs in mA (*not* A)."
    end

    # Group lines together ensuring that no λ is closer to it's neighbour than twice the ew_window_size.
    group_indices = linelist_neighbourhood_indices(linelist, ew_window_size)

    d_A = Array{Float64}(undef, length(linelist))
    for indices in group_indices
        wl_ranges = map(linelist[indices]) do line
            λ_start, λ_stop = (1e8 * line.wl - ew_window_size, 1e8 * line.wl + ew_window_size)
            wls = range(λ_start, λ_stop; length=Int(round((λ_stop - λ_start)/λ_step))+1)
        end

        spectrum = Korg.synthesize(
            atm, linelist[indices], A_X, wl_ranges;
            hydrogen_lines=false,
            synthesize_kwargs...
        )

        for (i, (idx, line)) in enumerate(zip(spectrum.subspectra, linelist[indices]))
            depth = 1 .- spectrum.flux[idx] ./ spectrum.cntm[idx]
            ew = trapz(spectrum.wavelengths[idx], depth) # Angstrom
            rew = log10(ew / (line.wl * 1e8))    
            d_A[indices[i]] = rew - A_X[Korg.get_atoms(line.species)[1]] # species is atomic
        end
    end

    # measured EWs are in mA, factor of 10^11 converts from cm
    measured_REW = log10.(ews ./ [1e11 * line.wl for line in linelist])
    measured_REW .- d_A
end

ews_to_abundances

In [60]:
include("Sun.jl")

281-element Vector{Korg.Line{Float64, Float64, Float64, Float64, Float64, Float64}}:
 Fe I 4365.9 Å (log gf = -2.25)
 Fe I 4389.25 Å (log gf = -4.58)
 Fe I 4445.47 Å (log gf = -5.44)
 Ti I 4465.8 Å (log gf = -0.16)
 Ti II 4470.86 Å (log gf = -2.06)
 Fe II 4508.29 Å (log gf = -2.52)
 Ca I 4512.27 Å (log gf = -1.9)
 Fe II 4520.22 Å (log gf = -2.65)
 Ti II 4544.03 Å (log gf = -2.53)
 Mg I 4571.1 Å (log gf = -5.62)
 ⋮
 O I 7775.39 Å (log gf = 0.0)
 Ni I 7797.59 Å (log gf = -0.34)
 Al I 7835.31 Å (log gf = -0.68)
 Al I 7836.13 Å (log gf = -0.45)
 Cu I 7933.13 Å (log gf = -0.37)
 S I 8693.93 Å (log gf = -0.44)
 S I 8694.62 Å (log gf = 0.1)
 Al I 8772.87 Å (log gf = -0.38)
 Al I 8773.9 Å (log gf = -0.22)

In [72]:
Teff, logg, Fe_H, vmic = (5500, 4.40, 0.0, 1.0)
A_X = Korg.format_A_X(Fe_H, 0.0)
atm = Korg.interpolate_marcs(Teff, logg, Fe_H)  

Korg.PlanarAtmosphere{Float64, Float64, Float64, Float64, Float64} with 56 layers

In [73]:
abundances = ews_to_abundances(atm, linelist, A_X, ews, vmic=vmic)

281-element Vector{Float64}:
 7.372571117745415
 7.382871767967065
 7.334091858805466
 4.868233978031333
 4.9348870290779
 7.336514809037024
 5.642299845121448
 7.480880656902284
 4.9729665141190385
 7.473793790277254
 ⋮
 8.302990306100924
 6.18691577660297
 5.955288830645784
 6.04051879749025
 4.203081953017105
 6.715626995751011
 7.074630587873986
 6.090918796397232
 6.154644721684386

In [74]:
# output
open("Sun.dat", "w") do fp
    write(fp, "species,Z,A,Sun\n")
    for (line, A) in zip(linelist, abundances)
        Z = string(Korg.get_atoms(line.species)[1])
        Sun = Korg.default_solar_abundances[parse(Int, Z)]
        write(fp, "$(line.species),$(Z),$(A),$(Sun)\n")
    end
end